# Problem 4

In this problem, we simply finetune a BERT model (not pretrained) on RTE dataset, and then finetune a BERT model (pretrained) on RTE dataset.

**IMPORTANT NOTES**:
- Please make sure that you have already read the part of hw5 pdf that corresponds to this problem. This is very important.
- At the end of the hw5, you will need to submit a zip folder containing three things. The instruction is also included in the first paragraph of the hw5 pdf.
  - (1) The writeup pdf containing your solutions to Problems 1, 2, 3, 4, 5. Yes, there're things you need to respond in your writeup (see hw5 pdf).
  - (2) The downloaded colab corresponding to Problem 4.
  - (3) The downloaded colab corresponding to Problem 5.

Some imports and data downloading

In [1]:
!git clone https://github.com/huggingface/transformers
!python transformers/utils/download_glue_data.py --tasks RTE
!pip install transformers=='3.5.1'

Cloning into 'transformers'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 54602 (delta 29), reused 33 (delta 3), pack-reused 54542
Receiving objects: 100% (54602/54602), 40.75 MiB | 20.14 MiB/s, done.
Resolving deltas: 100% (38161/38161), done.
	Completed!
     |████████████████████████████████| 1.3MB 13.0MB/s 
     |████████████████████████████████| 2.9MB 63.1MB/s 
     |████████████████████████████████| 1.1MB 56.0MB/s 
     |████████████████████████████████| 890kB 56.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=eb46b5a8ee31679afac37617052425fc248d9802ee986a3456067f361cf39d2b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

import numpy as np

import torch
import torch.nn as nn 
from transformers import AutoTokenizer, EvalPrediction, GlueDataset, GlueDataTrainingArguments, AutoModel, BertPreTrainedModel, AutoConfig, BertModel
from transformers import GlueDataTrainingArguments 
from transformers import (
    Trainer,
    TrainingArguments,
    glue_compute_metrics,
    glue_tasks_num_labels,
    set_seed,
)

# Import statistics for mean and variance calculation
from statistics import *

In [3]:
!ls glue_data/RTE/

dev.tsv  test.tsv  train.tsv


In [4]:
model_name = "bert-base-uncased"

data_args = GlueDataTrainingArguments(task_name="rte", data_dir="./glue_data/RTE")
training_args = TrainingArguments(
    logging_steps=50, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=64, 
    save_steps=1000,
    evaluate_during_training=True,
    output_dir="./models/rte",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    learning_rate=0.00001,
    num_train_epochs=15,
)
set_seed(42)
num_labels = glue_tasks_num_labels[data_args.task_name]

/usr/local/lib/python3.6/dist-packages/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,


### From non-pretrained BERT

TODO:
- Complete the following three lines such that ```tokenizer``` and ```config``` and ```bert_model``` corresponds to the ```model_name``` we defined in the above cells. 
- IMPORTANT: make sure that the BERT model does not load pretrained weights!
- Hint: https://huggingface.co/transformers/model_doc/auto.html and other relevant Hugging Face documentations. Consider using the tools we imported in the first cell. More hints: it's okay to use ```from_pretrained``` in the first two lines, depending on what class you use.

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
bert_model = AutoModel.from_config(config)
## Model configuration class with all the parameters of the model. 
## Initializing with a config file does not load the weights associated with the model, 
## only the configuration.
## Check out the from_pretrained() method to load the model weights.

TODO:
- Complete the forward function of the following class such that the model can do finetuning on RTE dataset.
- For more instructions, please refer to the hw5 pdf.

In [6]:
import math
class SequenceClassificationBERT(nn.Module):
      
    def __init__(self, config, bert_model):
        super().__init__()
        self.config = config
        self.num_labels = config.num_labels

        # Adding linear layer and Tanh nonlinearity
        self.linear = nn.Linear(config.hidden_size, config.hidden_size)
        self.act = nn.Tanh()

        self.softmax = nn.Softmax()
        self.loss = nn.BCELoss()
        
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.bert = bert_model

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # make sure that all the arguments in the forward() function is used
        # somewhere in the code

        ##### 

        # TODO

        #####

        outputs= self.bert(input_ids= input_ids, attention_mask= attention_mask, token_type_ids= token_type_ids, position_ids= position_ids, head_mask= head_mask, inputs_embeds= inputs_embeds, output_attentions= output_attentions, output_hidden_states= output_hidden_states, return_dict= return_dict)
        last_hidden_layer = outputs[0]
        h_CLS = last_hidden_layer[:, 0, :]
        x = self.linear(h_CLS)
        x = self.act(x)
        x = self.dropout(x)
        logits = self.classifier(x)

        soft_output = self.softmax(logits)
        soft_output, _= torch.max(soft_output, dim= 1)
        soft_output = soft_output.float()
        labels = labels.float()
        loss = self.loss(soft_output, labels)

        # do not change the lines below, so make sure your code works for the
        # lines below
        
        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

In [7]:
model = SequenceClassificationBERT(config=config, bert_model=bert_model)

TODO:
- Print out the number of trainable parameters in the BERT model. This can be done in one line. Please feel free to look up resources online. We also briefly touched upon relevant materials in Lab 3, but here, make sure you only count the number of trainable parameters.

In [8]:
 n_parameters = sum(p.numel() for p in model.parameters()) - bert_model.num_parameters() # TODO
 print(n_parameters)

592130


In [9]:
train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/glue.py:77: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:521: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with t

Now we train the model. Please make sure to read the pdf instructions. When you report results in the pdf writeup, make sure you report the mean and std of >=3 runs with different random seeds. Consider using ```set_seed(some number)``` before the below cell, before each run.

Make sure in each run, you're picking the best validation accuracy. We're using Trainer instead of the normal training loop which we have seen many many times earlier in the semester. In the trainer, we need to specify ```num_train_epochs``` (in ```training_args```) which we defined above. Please feel free to modify ```training_args``` such that:
- The learning rate is small (around 0.00001).
- Your model doesn't have large improments on validation accuracy anymore, at the end of training. The expected behavior is that the final validation accuracy won't be much better than chance.

We provided part of an example log below, but you may be able to get better accuracy. Again, make sure this run corresponds to using an non-pretrained BERT.

In [10]:
seed_values = {50, 100, 200}
best_accuracies = []
best_accuracy = -1
def compute_metrics(p: EvalPrediction):
    global best_accuracy
    preds = np.argmax(p.predictions, axis=1)
    data = glue_compute_metrics(data_args.task_name, preds, p.label_ids)
    if data['acc'] > best_accuracy:
      best_accuracy = data['acc']
    return data

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

for idx, seed_value in enumerate(seed_values):
  set_seed(seed_value)
 
  trainer.train()
  trainer.evaluate()
  best_accuracies.append(best_accuracy)
  best_accuracy = -1

print("-------Mean and Variance of best accuracies of all runs-------")
print(best_accuracies)
print(mean(best_accuracies))
print(variance(best_accuracies))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Step,Training Loss,Validation Loss,Acc
50,0.735441,0.765822,0.472924
100,0.707512,0.738659,0.527076
150,0.703886,0.695576,0.537906
200,0.711489,0.738928,0.472924
250,0.704179,0.695259,0.472924
300,0.702835,0.704539,0.527076
350,0.701514,0.706071,0.472924
400,0.698400,0.707357,0.527076
450,0.700822,0.696187,0.472924
500,0.701418,0.694245,0.527076


/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument

Step,Training Loss,Validation Loss,Acc
50,0.697717,0.693323,0.469314
100,0.698149,0.693323,0.469314
150,0.695276,0.693323,0.469314
200,0.697689,0.693323,0.469314
250,0.695742,0.693323,0.469314
300,0.694533,0.693323,0.469314
350,0.697849,0.693323,0.469314
400,0.696776,0.693323,0.469314
450,0.697208,0.693323,0.469314
500,0.695662,0.693323,0.469314


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning

Step,Training Loss,Validation Loss,Acc
50,0.698409,0.693323,0.469314
100,0.694123,0.693323,0.469314
150,0.696060,0.693323,0.469314
200,0.696519,0.693323,0.469314
250,0.696907,0.693323,0.469314
300,0.694736,0.693323,0.469314
350,0.696748,0.693323,0.469314
400,0.696122,0.693323,0.469314
450,0.696710,0.693323,0.469314
500,0.695240,0.693323,0.469314


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning

-------Mean and Variance of best accuracies of all runs-------
[0.5379061371841155, 0.4693140794223827, 0.4693140794223827]
0.49217809867629364
0.001568290129329631


### From pretrained BERT

Now, let's do the above experiments using a pretrained BERT!

TODO:
- Complete the following three lines such that ```tokenizer``` and ```config``` and ```bert_model``` corresponds to the ```model_name``` we defined in the above cells. 
- IMPORTANT (different from the TODO a few cells above): make sure that the BERT model below loads the pretrained weights!

In [11]:
tokenizer =  AutoTokenizer.from_pretrained(model_name)# TODO
config = AutoConfig.from_pretrained(model_name)# TODO
bert_model = AutoModel.from_pretrained(model_name)# TODO

In [12]:
bert_model 

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [13]:
train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/glue.py:77: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:521: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


In [14]:
model = SequenceClassificationBERT(config=config, bert_model=bert_model)

TODO:
- Similarly, we train the model. For more instructions, please see the TODO cells above (i.e., the TODO corresponding to training the model, when we're not loading weights into BERT), as well as the hw5 pdf.

In [15]:
seed_values = {50, 100, 200}
best_accuracies = []
best_accuracy = -1
def compute_metrics(p: EvalPrediction):
    global best_accuracy
    preds = np.argmax(p.predictions, axis=1)
    data = glue_compute_metrics(data_args.task_name, preds, p.label_ids)
    if data['acc'] > best_accuracy:
      best_accuracy = data['acc']
    return data

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

for idx, seed_value in enumerate(seed_values):
  set_seed(seed_value)
 
  trainer.train()
  trainer.evaluate()
  best_accuracies.append(best_accuracy)
  best_accuracy = -1

print("-------Mean and Variance of best accuracies of all runs-------")
print(best_accuracies)
print(mean(best_accuracies))
print(variance(best_accuracies))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Step,Training Loss,Validation Loss,Acc
50,0.695086,0.696462,0.472924
100,0.694259,0.698012,0.490975
150,0.687108,0.694298,0.440433
200,0.669967,0.693654,0.433213
250,0.657406,0.689275,0.465704
300,0.631711,0.683603,0.429603
350,0.610125,0.703673,0.422383
400,0.584948,0.710148,0.404332
450,0.554391,0.729400,0.404332
500,0.543624,0.744713,0.400722


/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument

Step,Training Loss,Validation Loss,Acc
50,0.447789,0.806134,0.382671
100,0.447239,0.806134,0.382671
150,0.439706,0.806134,0.382671
200,0.450964,0.806134,0.382671
250,0.451671,0.806134,0.382671
300,0.445887,0.806134,0.382671
350,0.439115,0.806134,0.382671
400,0.453491,0.806134,0.382671
450,0.449185,0.806134,0.382671
500,0.444408,0.806134,0.382671


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning

Step,Training Loss,Validation Loss,Acc
50,0.448658,0.806134,0.382671
100,0.442014,0.806134,0.382671
150,0.449910,0.806134,0.382671
200,0.452827,0.806134,0.382671
250,0.441127,0.806134,0.382671
300,0.444294,0.806134,0.382671
350,0.455822,0.806134,0.382671
400,0.443157,0.806134,0.382671
450,0.440644,0.806134,0.382671
500,0.438437,0.806134,0.382671


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning

-------Mean and Variance of best accuracies of all runs-------
[0.49097472924187724, 0.38267148014440433, 0.38267148014440433]
0.4187725631768953
0.003909864588356422
